In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import gseapy as gp
import pdb
import copy
from scipy import sparse
import anndata
import cerberus
import subprocess

p = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
sys.path.append(p)

from scripts.utils import *
from scripts.plotting import *

In [2]:
major_set = '../swan/isos_sample_gene_90.tsv'
gtf = '../cerberus.gtf'

## Get GTF for only the major isoforms for a given gene

In [3]:
gene = 'CTCF'

In [4]:
def get_gene_major_iso_gtf(gene, gtf, major_set):
    ofile = '{}_major_isos.gtf'.format(gene.lower())
    df = pd.read_csv(major_set, sep='\t')
    df = df.loc[df.gname == gene]
    tids = df.tid.unique().tolist()
    tid_str = "'"
    for i, tid in enumerate(tids):
        if i == 0: 
            tid_str += tid
        else:
            tid_str += '|'+tid
    tid_str += "'"
    grep_cmd = 'grep -E {} {} > {}'.format(tid_str, gtf, ofile)
    grep_cmd = grep_cmd.replace('[', '\[')
    grep_cmd = grep_cmd.replace(']', '\]')
    # print(grep_cmd)
    # grep_cmd = grep_cmd.split()
    # print(grep_cmd)
    # subprocess.run(grep_cmd, check=True)
    print(grep_cmd)
    


In [5]:
gene = 'CTCF'
get_gene_major_iso_gtf(gene, gtf, major_set)

grep -E 'ENSG00000102974\[3,5,1\]|ENSG00000102974\[1,1,1\]|ENSG00000102974\[9,16,1\]|ENSG00000102974\[8,14,1\]|ENSG00000102974\[12,16,1\]|ENSG00000102974\[10,17,1\]|ENSG00000102974\[1,6,1\]|ENSG00000102974\[11,5,1\]|ENSG00000102974\[1,18,5\]|ENSG00000102974\[1,19,6\]' ../cerberus.gtf > ctcf_major_isos.gtf


In [6]:
gene = 'ELN'
get_gene_major_iso_gtf(gene, gtf, major_set)

grep -E 'ENSG00000049540\[1,112,3\]|ENSG00000049540\[2,2,2\]|ENSG00000049540\[1,137,3\]|ENSG00000049540\[1,8,3\]|ENSG00000049540\[1,2,3\]|ENSG00000049540\[1,157,1\]|ENSG00000049540\[1,14,3\]|ENSG00000049540\[1,135,3\]|ENSG00000049540\[1,76,3\]|ENSG00000049540\[1,77,3\]|ENSG00000049540\[1,1,1\]|ENSG00000049540\[2,1,1\]|ENSG00000049540\[1,69,1\]|ENSG00000049540\[1,108,3\]|ENSG00000049540\[1,111,3\]|ENSG00000049540\[1,1,3\]|ENSG00000049540\[1,8,1\]|ENSG00000049540\[1,16,4\]|ENSG00000049540\[1,76,1\]|ENSG00000049540\[1,96,3\]|ENSG00000049540\[1,2,1\]|ENSG00000049540\[1,57,3\]|ENSG00000049540\[1,3,3\]|ENSG00000049540\[1,147,3\]|ENSG00000049540\[1,77,1\]|ENSG00000049540\[1,98,3\]|ENSG00000049540\[1,136,1\]|ENSG00000049540\[1,147,1\]|ENSG00000049540\[1,112,1\]|ENSG00000049540\[1,135,1\]|ENSG00000049540\[1,109,1\]|ENSG00000049540\[1,13,3\]|ENSG00000049540\[1,149,3\]|ENSG00000049540\[2,36,3\]|ENSG00000049540\[1,86,3\]|ENSG00000049540\[1,4,3\]|ENSG00000049540\[1,14,1\]|ENSG00000049540\[1,138,3\]